In [ ]:
# Install all dependencies for sgementation-models-3D library.
# We will use this library to call 3D unet.
# Alternative, you can define your own Unet, if you have skills!
!pip install classification-models-3D
!pip install efficientnet-3D
!pip install segmentation-models-3D

# Use patchify to break large volumes into smaller for training 
#and also to put patches back together after prediction.
!pip install patchify

In [ ]:
import os
import random
import logging
import h5py
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

# funky librairies for segmentation
import segmentation_models_3D as sm
from patchify import patchify, unpatchify

print('All librairies sucessfully imported.')

In [ ]:
# import data
PATH_COLAB = '/content/drive/MyDrive/6_aneurysm_segmentation/challenge_dataset.zip'
PATH_DEVICE = './challenge_dataset/'

try:
    from google.colab import drive
    logging.info('Working on Colab.')
    
    # connect your drive to the session
    drive.mount('/content/drive')

    %cd /content/drive/MyDrive/6_aneurysm_segmentation/

    # unzip data into the colab session
    ! unzip $PATH_COLAB -d /content
    logging.info('Data unziped in your Drive.')

    %cd ../../..

except:
    logging.info('Working on your device.')
    
    data_exists = os.path.exists(PATH_DEVICE)
    
    if data_exists:
        logging.info(f"Dataset found on device at : '{PATH_DEVICE}.'") 
    else:
        raise FileNotFoundError(f"Data folder not found at '{PATH_DEVICE}'")

# Get data

In [ ]:
PATH_DATASET='./challenge_dataset/'
TEST_SIZE = 0.2 # % of test samples from the full dataset
VAL_SPLIT = 0.2 # % of training samples kept for the validation metrics
CROP = 64

In [ ]:
# get file names
file_names = os.listdir(PATH_DATASET)
N = len(file_names)
print(f'{N} samples in dataset.')

# open all .h5 files, split inputs and target masks, store all in np.arrays
raw_data = []
labels = []
names = []

for file_name in tqdm(file_names):
    f = h5py.File(f'{PATH_DATASET}/{file_name}', 'r')

    X, Y = np.array(f['raw']), np.array(f['label'])

    X = X[:,CROP:2*CROP,CROP:2*CROP]
    Y = Y[:,CROP:2*CROP,CROP:2*CROP]

    raw_data.append(X)
    labels.append(Y)
    names.append(file_name)

    # TO KEEP FOR LATER - USEFUL TO ADD FREE SAMPLES BY CROPPING
    # X_patches = patchify(X, (64, 64, 64), step=64)  # Step=64 for 64 patches means no overlap
    # X_patches_resh = np.reshape(X_patches, (-1, X_patches.shape[3], X_patches.shape[4], X_patches.shape[5]))
    # Y_patches = patchify(Y, (64, 64, 64), step=64)  # Step=64 for 64 patches means no overlap
    # Y_patches_resh = np.reshape(Y_patches, (-1, Y_patches.shape[3], Y_patches.shape[4], Y_patches.shape[5]))
    # raw_data.append(X_patches_resh)
    # labels.append(Y_patches_resh)
    # names.append(file_name)

# convert to arrays for patchify
raw_data = np.array(raw_data)
labels = np.array(labels)

# raw_data = np.reshape(raw_data, (-1, raw_data.shape[2], raw_data.shape[3], raw_data.shape[4]))
# labels = np.reshape(labels, (-1, labels.shape[2], labels.shape[3], labels.shape[4]))

raw_data = np.stack((raw_data,) * 3, axis=-1)
labels = np.expand_dims(labels, axis=4)

# check shapes
print(raw_data.shape)
print(labels.shape)

In [ ]:
SCAN_ID = 45
DEPTH = 32

fig, ax = plt.subplots(1, 2)
ax[0].imshow(raw_data[SCAN_ID,:,:,DEPTH,0])
ax[1].imshow(labels[SCAN_ID,:,:,DEPTH,0]) # last 0 to get a 2D image
plt.show()

In [ ]:
# split train and test data
X_train, X_test, y_train, y_test = train_test_split(raw_data, labels, test_size=TEST_SIZE)

# train_test_split returns lists, we want arrays for easier calls
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train, dtype='float32')
y_test = np.array(y_test, dtype='float32')

# check shapes
print(X_train.shape)
print(X_test.shape)

# Generator

In [ ]:
import os
import numpy as np
import h5py

from tensorflow.keras.preprocessing.image import ImageDataGenerator

class AugmentedGenerator:
    '''
    Example Usage:
    >>> augmented_generator = AugmentedGenerator()
    >>> train_generator = augmented_generator.train_generator
    >>> validation_generator = augmented_generator.validation_generator
    >>> model.fit(train_generator, validation_data=validation_generator, epochs=10)
    '''
    def __init__(self, path='./challenge_dataset/', validation_split=0.2, batch_size=16, seed=1):
        self.path = path
        self.validation_split = validation_split
        self.batch_size = batch_size
        self.seed = seed
        self.raw = None
        self.labels = None
        self._load_data()
        self.train_generator = self._get_train_generator()
        self.validation_generator = self._get_validation_generator()
    
    def _load_data(self):
        '''
        Loads the data from the .h5 files, might be better to use a generator here as well.
        '''
        data_exists = os.path.exists(self.path)

        if not data_exists:
            raise FileNotFoundError(f"Data folder not found at '{self.path}'")
        else:
            l_raw = []
            l_label = []
            for i in range(103): # might be better to make it cleaner
                f = h5py.File(self.path + f'scan_{i+1}.h5', 'r')
                l_raw.append(f['raw'])
                l_label.append(f['label'])
            self.raw, self.labels = np.array(l_raw), np.array(l_label)

    def _get_train_generator(self):
        '''
        Returns a generator for the training data with data augmentation.
        '''
        data_gen_params = dict(
            validation_split=self.validation_split,
            rotation_range=90,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            data_format='channels_first'
        )
        data_gen = ImageDataGenerator(**data_gen_params)
        
        return data_gen.flow(self.raw, self.labels, batch_size=self.batch_size, subset='training', seed=self.seed)


    def _get_validation_generator(self):
        '''
        Returns a generator for the validation data : no data augmentation.
        '''
        data_gen = ImageDataGenerator(
            validation_split=self.validation_split,
            data_format='channels_first')
        return data_gen.flow(self.raw, self.labels, batch_size=self.batch_size, subset='validation', seed=self.seed)


In [ ]:
# get file names in the folder
PATH_DATASET = 'challenge_dataset'

file_names = os.listdir(PATH_DATASET)
N = len(file_names)

VAL_SPLIT = 0.4
BATCH_SIZE = 6

generator = AugmentedGenerator(validation_split=VAL_SPLIT, batch_size=BATCH_SIZE)

In [ ]:
train_data_pipeline, val_data_pipeline = generator.train_generator, generator.validation_generator

In [ ]:
def crop(generator, crop_size = CROP, preprocess = lambda x:x, channels = 3):
    while True:
        x, y = next(generator)

        crop_x, crop_y = np.zeros((len(x), crop_size, crop_size, crop_size)), np.zeros((len(x), crop_size, crop_size, crop_size))

        for i in range(len(x)):
            aneurysm_indexes = np.nonzero(y[i])

            z_min, z_max = aneurysm_indexes[0].min(), aneurysm_indexes[0].max()
            x_min, x_max = aneurysm_indexes[1].min(), aneurysm_indexes[1].max()
            y_min, y_max = aneurysm_indexes[2].min(), aneurysm_indexes[2].max()

            #print(z_min, z_max, x_min, x_max, y_min, y_max)

            z_init = np.random.randint(max(0, z_max - crop_size), max(max(0, z_max - crop_size) + 1, min(z_min + 1, x.shape[1] - crop_size)))
            y_init = np.random.randint(max(0, y_max - crop_size), max(max(0, y_max - crop_size) + 1, min(y_min + 1, x.shape[3] - crop_size)))
            x_init = np.random.randint(max(0, x_max - crop_size), max(max(0, x_max - crop_size) + 1, min(x_min + 1, x.shape[2] - crop_size)))

            #print(x.shape)

            crop_x[i] = x[i,z_init: z_init + crop_size, x_init: x_init + crop_size, y_init : y_init + crop_size]
            crop_y[i] = y[i,z_init: z_init + crop_size, x_init: x_init + crop_size, y_init : y_init + crop_size]


        

        
        yield(preprocess(np.stack((crop_x,)*channels, axis=-1)), crop_y)

In [ ]:
crop_train = crop(train_data_pipeline)
crop_val = crop(val_data_pipeline)

In [ ]:
next(crop_train)[0].shape

# Models

In [ ]:
# Loss Function and coefficients to be used during training:
def custom_iou(smooth=1e-6):
    """
    Returns a IoU function, with a custom smoothing parameter.
    Such a double function is needed because loss function in Keras are expected
    to take only two parameters. Therefore, smooth couldn't be a parameter.
    May be removed in future commits beacuse seems finally useless.
    """
    def IoULoss(targets, inputs):
        """
        Returns the intersection over union (IoU) of the two inputs masks.
        """
        inputs = tf.cast(inputs, tf.float32)
        targets = tf.cast(targets, tf.float32)
        # flatten label and prediction tensors
        inputs = K.flatten(inputs)
        targets = K.flatten(targets)

        intersection = K.sum(targets * inputs)
        total = K.sum(targets) + K.sum(inputs)
        union = total - intersection
        
        IoU = (intersection + smooth) / (union + smooth)
        return 1 - IoU

    return IoULoss

Backbones: ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'seresnet18', 'seresnet34', 'seresnet50', 'seresnet101', 'seresnet152', 'seresnext50', 'seresnext101', 'senet154', 'resnext50', 'resnext101', 'vgg16', 'vgg19', 'densenet121', 'densenet169', 'densenet201', 'inceptionresnetv2', 'inceptionv3', 'mobilenet', 'mobilenetv2', 'efficientnetb0', 'efficientnetb1', 'efficientnetb2', 'efficientnetb3', 'efficientnetb4', 'efficientnetb5', 'efficientnetb6', 'efficientnetb7']

So far, what worked best for our dataset (IOU_test = 43%) is:
```
BACKBONE = 'resnet50'
LOSS_TYPE = 'jaccard'
BATCH_SIZE = 8
LR = 1e-4
```

In [ ]:
# MODEL PARAMETERS
encoder_weights = 'imagenet' # Try 'imagenet' or None (random initialization)
BACKBONE = 'resnet50'  # Try vgg16, efficientnetb7, inceptionv3, resnet50
activation = 'sigmoid' # final layer activation function, sigmoid for binary
patch_size = 64 # cube side length
n_classes = 1 # num channels output, here binary segmentation so 1
channels = 3 # num channels input, need 3 because backbones are trained on RGB
LOSS_TYPE = 'jaccard' # check dict 'losses' down below

# TRAINING PARAMETERS
LR = 1e-4 # starting learning rate
EPOCHS = 100
BATCH_SIZE = 12

MODEL_NAME = f'./3D_model_{BACKBONE}_{encoder_weights}weights_{EPOCHS}epochs_{LOSS_TYPE}'
print(MODEL_NAME)

In [ ]:
# Define optimizer
optim = tf.keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
dice_loss = sm.losses.DiceLoss() 
jaccard_loss = sm.losses.JaccardLoss()
focal_loss = sm.losses.BinaryFocalLoss()

losses = {'dice': dice_loss,
          'jaccard': jaccard_loss,
          'custom_jaccard': custom_iou(),
          'focal_loss': focal_loss,
          'dice_focal': dice_loss + (1 * focal_loss), 
          'jaccard_focal': jaccard_loss + (1 * focal_loss), 
          }

total_loss = losses.get(LOSS_TYPE)
assert total_loss is not None, ('Loss not defined. Check your spelling of LOSS_TYPE or the dict losses.')

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
# add callbacks to monitor training 
weight_path = "{}_weights.best.hdf5".format(MODEL_NAME)

checkpoint = ModelCheckpoint(weight_path, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min', 
                             save_weights_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', 
                                   factor=0.5, 
                                   patience=4, 
                                   verbose=1, 
                                   mode='auto', 
                                   min_delta=0.0001, 
                                   cooldown=5, 
                                   min_lr=1e-6)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=12) 

callbacks_list = [checkpoint, 
                  early, 
                  reduceLROnPlat]

In [ ]:
# Preprocess input data - otherwise you end up with garbage resutls 
# and potentially model that does not converge.
preprocess_input = sm.get_preprocessing(BACKBONE)

X_train_prep = preprocess_input(X_train)
X_test_prep = preprocess_input(X_test)

In [ ]:
# Define the model. Here we use Unet but we can also use other model architectures from the library.
model = sm.Unet(BACKBONE, classes=n_classes, 
                input_shape=(patch_size, patch_size, patch_size, channels), 
                encoder_weights=encoder_weights,
                activation=activation)

model.compile(optimizer = optim, loss=total_loss, metrics=metrics)
# print(model.summary())

In [ ]:
val_steps = int(N * VAL_SPLIT) // BATCH_SIZE
train_steps = int(N * (1 - VAL_SPLIT)) // BATCH_SIZE

crop_train = crop(train_data_pipeline, preprocess=preprocess_input)
crop_val = crop(val_data_pipeline, preprocess=preprocess_input)

history = model.fit(crop_train, 
            steps_per_epoch=train_steps,
            epochs=400,
            validation_data=crop_val,
            validation_steps=val_steps,
            callbacks=callbacks_list)

In [ ]:
# Fit the model
#history = model.fit(X_train_prep, 
#                    batch_size=BATCH_SIZE, 
#                    y_train,
#                    epochs=EPOCHS,
#                    verbose=1,
#                    validation_split=VAL_SPLIT,
#                    callbacks=callbacks_list)

In [ ]:
# plot the training and validation IoU and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['iou_score']
val_acc = history.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and validation IOU')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

# Predict

In [ ]:
# Load the pretrained model for testing and predictions. 
# you need a model instance before loading weights
print(f"Reload weights from : {MODEL_NAME}_weights.best.hdf5")
model.load_weights(f"{MODEL_NAME}_weights.best.hdf5")

In [ ]:
# Predict on the test data
y_pred = model.predict(X_test_prep)

In [ ]:
THRESHOLD = 0.5

y_pred01 = (y_pred > THRESHOLD).astype(int) # float => boolean => binary (0/1)

print(f'------ AFTER THRESHOLDING AT {THRESHOLD} ------')
print('> sm.metrics.IOUScore :', sm.metrics.IOUScore()(y_test, y_pred01).numpy())
print('> Custom IoU :', 1 - custom_iou()(y_test, y_pred01).numpy()) # to check my custom function, it was a loss so 1 - loss

# precision_recall_fscore_support report
precision, recall, fscore, support = precision_recall_fscore_support(y_test.flatten(), 
                                                                  y_pred01.flatten()) 
print('> Precision :', precision[1])
print('> Recall :', recall[1])
print('> Fscore :', fscore[1])

# Confusion matrix
cm = confusion_matrix(y_test.flatten(), 
                      y_pred01.flatten())
print('\nConfusion matrix :\n', cm)

In [ ]:
# Test some random images

# pick a random test scan and its ground truth mask
test_img_number = random.randint(0, len(X_test)-1)
print(f'I choose test image n° {test_img_number} ...')

test_img = X_test[test_img_number]
ground_truth = y_test[test_img_number]

# process input image before prediction
test_img_input = np.expand_dims(test_img, 0)
test_img_input1 = preprocess_input(test_img_input)

# prediction
test_pred = model.predict(test_img_input1)
test_pred = test_pred.squeeze()

# thresholding + reshaping
print(test_pred.shape)

In [ ]:
# Plot individual slices from test predictions for verification
SLICE_MIN, SLICE_MAX = 25, 45

for slice in range(SLICE_MIN, SLICE_MAX+1):
    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title(f'Testing Image {slice}')
    plt.imshow(test_img[slice,:,:,0], cmap='gray')
    plt.subplot(232)
    plt.title(f'Testing Label {slice}')
    plt.imshow(ground_truth[slice,:,:,0])
    plt.subplot(233)
    plt.title(f'Prediction on test image {slice}')
    plt.imshow(test_pred[slice,:,:])
    plt.show()